# After cloning:
* .h5 might be corrupt. If so, delete it and redownload it.<br>Original weights from: https://github.com/pedropro/TACO/releases/tag/1.0
<br>Custom weights from: https://github.com/abbesmoe/TrashDetection
* upload original images into TrashDetection/static/uploads directory



In [ ]:
# Clone github once
!git clone https://github.com/abbesmoe/TrashDetection

In [ ]:
# Move github to "/MyDrive". This helps you not lose any progress
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
# Install latest versions
!pip install keras==2.6.0
!pip install tensorflow==2.6.0
!pip install 'h5py==2.10.0'

In [ ]:
# Change directory to /TrashDetection
%cd /content/gdrive/MyDrive/TrashDetection/

In [ ]:
# Import packages
%matplotlib inline
import csv
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import skimage.io
from detector import dataset
from detector import model as modellib
from detector import visualize
from detector.config import Config
from os import path
from PIL import Image

In [ ]:
# Load class map
class_map = {}
with open("detector/taco_config/map_10.csv") as csvfile:
    reader = csv.reader(csvfile)
    class_map = {row[0]:row[1] for row in reader}

TACO_DIR = "data/"
round = None 
subset = "train"
dataset = dataset.Taco()
taco = dataset.load_taco(TACO_DIR, round, subset,
                         class_map=class_map, return_taco=True)

dataset.prepare()

print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
# Create config class and override hyperparameters
class TacoTestConfig(Config):
  NAME = "taco"
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1
  NUM_CLASSES = dataset.num_classes

  # Changed for training
  # BACKBONE = "resnet101"
  # USE_OBJECT_ZOOM = False
  # TRAIN_ROIS_PER_IMAGE = 128
  # LEARNING_RATE = 0.01
  # GRADIENT_CLIP_NORM = 10.0

config = TacoTestConfig()
config.display()

In [ ]:
# Create models and load weights
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_taco.hy',
                          config=config)

model.load_weights('mask_rcnn_taco_0100.h5', by_name=True,
                   weights_out_path=None)

In [ ]:
# Original classes (map_10)
class_names = ["BG","Bottle","Bottle cap","Can","Cigarette","Cup",
               "Lid","Other","Plastic bag + wrapper","Pop tab","Straw"]

# Custom classes
#class_names = ['BG', 'Bag & wrapper', 'Blister pack', 'Bottle cap',
#               'Can', 'Cardboard', 'Carton', 'Cigarette', 'Container',
#               'Cup', 'Food waste', 'Glass', 'Glass bottle', 'Lid',
#               'Other', 'Other plastic', 'Paper', 'Plastic bottle',
#               'Pop tab', 'Straw', 'Styrofoam', 'Tube']

In [ ]:
# Load uploaded image
imagePath = "static/uploads/"
imageName = "trash.jpg"
img = Image.open(os.path.join(imagePath, imageName))

fileName = os.path.splitext(imageName)[0]
fileExtension = os.path.splitext(imageName)[1]

# If file isn't jpg, save it as one
if fileExtension != '.jpg':
  img.convert('RGB').save("{}{}.jpg".format(imagePath, fileName))
  imageName = "{}.jpg".format(fileName)

image = skimage.io.imread(os.path.join(imagePath, imageName))

In [ ]:
# run detection
r = model.detect([image], verbose=0)[0]

In [ ]:
# Define minimum detection accuracy
def min_accuracy(r,a):
  result = {'rois': [], 'masks': [], 'class_ids': [], 'scores': []}
  indecies = []
  for i,ele in enumerate(r['scores']):
    if ele >= a:
      result['rois'].append(r['rois'][i])
      result['class_ids'].append(r['class_ids'][i])
      result['scores'].append(r['scores'][i])
      indecies.append(i)
  
  result['masks'] = r['masks'][:,:,indecies]

  result['rois'] = np.asarray(result['rois'])
  result['masks'] = np.asarray(result['masks'])
  result['class_ids'] = np.asarray(result['class_ids'])
  result['scores'] = np.asarray(result['scores'])
  return result

r = min_accuracy(r,0.7)
print(r['class_ids'])

In [ ]:
# Display original image
plt.figure(figsize=(12,10))
skimage.io.imshow(image)

In [ ]:
# Display annotated image
visualize.display_instances(image, imageName, r['rois'], r['masks'],
                            r['class_ids'], class_names, r['scores'])

# Blocks for filtering

In [ ]:
# run only once, unless you want to empty data.json
with open('data.json', 'w') as f:
  data = {"Images":[]}

In [ ]:
# Run after every detection to write data in json file
size = len(r['class_ids'])
i = 0
classNameList = []
while i < size:
  obj_name = class_names[r['class_ids'][i]]
  classNameList.append(obj_name)
  i = i + 1

def write_json(data, filename="data.json"):
  with open(filename, "w") as f:
    json.dump(data, f, indent=4)


img_data = {}
img_data["Name"] = "annotated_{}".format(imageName)
img_data["Quantity"] = len(classNameList)
img_data['Classes'] = classNameList

data['Images'].append(img_data)

write_json(data)

In [ ]:
# Display tables
pd.set_option("display.max_rows", None, "display.max_columns", None)

with open('data.json') as json_data:
    data = json.load(json_data)

df = pd.DataFrame(data['Images'])
print(df)